In [1]:
import pandas as pd
import numpy as np
import duckdb   

# Chargement des données

In [2]:
# General Path to the directory
# csv_path = r'C:\Users\gunon\Documents\bootcamp-main\7-SQL\9-Exercices SQL\Projet_wine/'
csv_path = 'https://github.com/gunony/Example_SQL_Data/raw/main/'

# Path to the databases
path_item = csv_path + 'item.csv'
path_client = csv_path + 'client.csv'
path_invoice = csv_path + 'invoice.csv'
#path_order = csv_path + 'wine_sales_2024_Order.csv'

# Load the tables
table_item = pd.read_csv(path_item)
table_client = pd.read_csv(path_client)
table_invoice = pd.read_csv(path_invoice, parse_dates=['date'])
#table_order = pd.read_csv(path_order, parse_dates=['date'])


Regarder la composition de la table des articles.\
La clé est le code_item.

In [3]:
query = """
SELECT *
FROM table_item
"""
duckdb.sql(query)

┌────────────┬─────────────────────────────────┬───────┬────────────┬────────────────┬─────────┐
│ code_item  │            item_name            │ year  │ base_price │     region     │  type   │
│  varchar   │             varchar             │ int64 │   double   │    varchar     │ varchar │
├────────────┼─────────────────────────────────┼───────┼────────────┼────────────────┼─────────┤
│ rpo2013100 │ Château L'Evangile 2013         │  2013 │      139.0 │ Pomerol        │ rouge   │
│ res2017100 │ Château Cos d'Estournel 2017    │  2017 │      150.0 │ Saint Estèphe  │ rouge   │
│ bsa2012100 │ Château Rayne Vigneau 2012      │  2012 │      137.0 │ Sauternes      │ blanc   │
│ bsa2018100 │ Château d'Yquem 2018            │  2018 │      169.0 │ Sauternes      │ blanc   │
│ res2021101 │ Château Calon-Ségur 2021        │  2021 │      120.0 │ Saint Estèphe  │ rouge   │
│ rma2021101 │ Château Giscours 2021           │  2021 │      157.0 │ Margaux        │ rouge   │
│ rpa2019100 │ Château Mouton 

Regarder la composition de la table des clients, dont la clé est le cust_id.

In [4]:
query = """
SELECT *
FROM table_client
"""
duckdb.sql(query)

┌───────────┬─────────────────────────────────────┬─────────────┬─────────────────────────────────────────┐
│ id_client │                town                 │   country   │               client_name               │
│  varchar  │               varchar               │   varchar   │                 varchar                 │
├───────────┼─────────────────────────────────────┼─────────────┼─────────────────────────────────────────┤
│ fr116     │ Annecy                              │ France      │ Le Clos des Sens                        │
│ fr118     │ Cassis                              │ France      │ La Villa Madie                          │
│ fr121     │ Courchevel                          │ France      │  Le 1947 à Cheval Blanc                 │
│ fr124     │ Île de Noirmoutier / L'Herbaudière  │ France      │ La Marine                               │
│ fr128     │ Menton                              │ France      │  Mirazur                                │
│ fr133     │ Paris         

Regarder la composition de la table des factures établies, dont la clé est le inv_no (numéro de facture).

In [5]:
query = """
SELECT *
FROM table_invoice
"""
duckdb.sql(query)

┌─────────────────────┬─────────┬────────────┬───────┬────────┬─────────┬──────────┐
│        date         │ cust_id │    item    │  qty  │ price  │ line_no │  inv_no  │
│    timestamp_ns     │ varchar │  varchar   │ int64 │ double │  int64  │ varchar  │
├─────────────────────┼─────────┼────────────┼───────┼────────┼─────────┼──────────┤
│ 2025-01-01 00:00:00 │ fr133   │ rpo1979100 │     3 │ 5320.0 │       1 │ fac01_01 │
│ 2025-01-03 00:00:00 │ fr141   │ rch2009100 │    10 │  210.0 │       1 │ fac01_02 │
│ 2025-01-05 00:00:00 │ fr116   │ rpo2013100 │     5 │  148.8 │       1 │ fac01_03 │
│ 2025-01-09 00:00:00 │ fr133   │ bsa2012100 │     7 │   39.7 │       1 │ fac01_04 │
│ 2025-01-10 00:00:00 │ fr140   │ bsa2018100 │     3 │  181.0 │       1 │ fac01_05 │
│ 2025-01-15 00:00:00 │ fr124   │ rma2021101 │     4 │  168.0 │       1 │ fac01_06 │
│ 2025-01-22 00:00:00 │ it236   │ rpa2019100 │    12 │  988.9 │       1 │ fac01_07 │
│ 2025-01-22 00:00:00 │ it236   │ res2017100 │    15 │  166.5 │  

Regarder la table des commandes avec comme colonne clé 'order_no' qui correspond au numéro de commande.

In [6]:

query = """
SELECT *
FROM table_order
LIMIT 10 
"""
# duckdb.sql(query)



# Les Jointures

### 1-Cross join
C'est un produit cartésien. Chaque ligne de la première table est combinée avec les lignes de la seconde table.

**Mission** : Affecter à chaque client la gamme complète des vins.

In [38]:
query = '''

SELECT
    id_client,
    client_name,
    code_item,
    item_name

FROM table_client

CROSS JOIN table_item
'''
duckdb.sql(query)

┌───────────┬──────────────────────┬────────────┬─────────────────────────────────┐
│ id_client │     client_name      │ code_item  │            item_name            │
│  varchar  │       varchar        │  varchar   │             varchar             │
├───────────┼──────────────────────┼────────────┼─────────────────────────────────┤
│ fr116     │ Le Clos des Sens     │ rpo2013100 │ Château L'Evangile 2013         │
│ fr116     │ Le Clos des Sens     │ res2017100 │ Château Cos d'Estournel 2017    │
│ fr116     │ Le Clos des Sens     │ bsa2012100 │ Château Rayne Vigneau 2012      │
│ fr116     │ Le Clos des Sens     │ bsa2018100 │ Château d'Yquem 2018            │
│ fr116     │ Le Clos des Sens     │ res2021101 │ Château Calon-Ségur 2021        │
│ fr116     │ Le Clos des Sens     │ rma2021101 │ Château Giscours 2021           │
│ fr116     │ Le Clos des Sens     │ rpa2019100 │ Château Mouton Rothschild 2019  │
│ fr116     │ Le Clos des Sens     │ bpe2020100 │ Château Smith Haut Lafitte

### 2-Inner join
Cette opération permet de combiner les lignes de deux tables en se basant sur une condition de jointure spécifique.\
La jointure renvoie uniquement les lignes où il y a une correspondance dans les deux tables.\
C'est un 'crossjoin' avec une condition de correspondance.\
Attention : S'il manque des informations dans une table, alors la ligne avec cette information manquante n'apparaître pas.

**Mission** : Ajouter dans la table des ventes le nom des clients et le libelle des produits.

In [39]:
query = '''
SELECT
    date,
    cust_id,
    client_name,
    item,
    item_name,
    inv_no
FROM 
    table_invoice
INNER JOIN
    table_client
ON 
    cust_id = id_client
INNER JOIN
    table_item
ON
    item = code_item
'''

duckdb.sql(query)


┌─────────────────────┬─────────┬─────────────────────────────────────────┬────────────┬─────────────────────────────────┬──────────┐
│        date         │ cust_id │               client_name               │    item    │            item_name            │  inv_no  │
│    timestamp_ns     │ varchar │                 varchar                 │  varchar   │             varchar             │ varchar  │
├─────────────────────┼─────────┼─────────────────────────────────────────┼────────────┼─────────────────────────────────┼──────────┤
│ 2025-01-01 00:00:00 │ fr133   │ L'Ambroisie                             │ rpo1979100 │ Pétrus 1979                     │ fac01_01 │
│ 2025-01-03 00:00:00 │ fr141   │ Assiette Champenoise                    │ rch2009100 │ Deutz 2009                      │ fac01_02 │
│ 2025-01-05 00:00:00 │ fr116   │ Le Clos des Sens                        │ rpo2013100 │ Château L'Evangile 2013         │ fac01_03 │
│ 2025-01-09 00:00:00 │ fr133   │ L'Ambroisie                 

### 4-Full outer join
C'est la combinaison des résultats d'un 'left join' et d'un 'right join'.\
Le résultat retourne toutes les lignes des deux tables avec des valeurs NULL lorsqu'il n'y a pas de correspondances.

**Mission** : Lister les clients qui n'ont pas été facturés sur la période. 


In [40]:
query = '''
SELECT 
    id_client,
    client_name,
    town,
    country,
    cust_id
FROM
    table_invoice
FULL OUTER JOIN
    table_client
ON
    cust_id = id_client
WHERE
    cust_id IS NULL
'''
duckdb.sql(query)

┌───────────┬────────────────────────┬───────────────────────┬────────────┬─────────┐
│ id_client │      client_name       │         town          │  country   │ cust_id │
│  varchar  │        varchar         │        varchar        │  varchar   │ varchar │
├───────────┼────────────────────────┼───────────────────────┼────────────┼─────────┤
│ ch109     │ Xin Rong Ji            │ Beijing               │ Chine      │ NULL    │
│ eu113     │ Smyth                  │ Chicago               │ Etats-Unis │ NULL    │
│ fr117     │ L'Oustau de Baumanière │ Les Baux-de-Provence  │ France     │ NULL    │
│ hk221     │ Caprice                │ Hong Kong             │ Hong Kong  │ NULL    │
└───────────┴────────────────────────┴───────────────────────┴────────────┴─────────┘

### 5-Self join
C'est une opération de jointure d'une table avec elle-même.\
Cette jointure nécessite d'utiliser des alias dans les commandes SELECT, FROM et JOIN pour distinguer les deux instances.

**Mission** : Ecrire une requête qui permet d'obtenir le nombre de client facturé la semaine présente et la semaine précédente (i.e. retained client).

In [41]:
query = """

-- créer une sous_table en provenance des ventes avec numéro client, nom et le no de la semaine--

WITH cte AS (
    SELECT 
        cust_id,
        WEEK(date) as week,
    FROM 
        table_invoice
    GROUP BY
        cust_id,
        week
    ORDER BY 
        week 
)

-- requete principale avec un SELF JOIN --
SELECT 
    l.week,
    COUNT(DISTINCT(l.cust_id)) AS retained_client
FROM 
    cte AS l
JOIN
    cte AS r
ON 
    l.cust_id = r.cust_id
    AND
    l.week = r.week + 1
GROUP BY 
    l.week
ORDER BY
    l.week

"""
duckdb.sql(query)

┌───────┬─────────────────┐
│ week  │ retained_client │
│ int64 │      int64      │
├───────┼─────────────────┤
│     2 │               1 │
│     8 │               2 │
│     9 │               1 │
│    10 │               1 │
│    13 │               1 │
│    15 │               1 │
│    16 │               2 │
│    19 │               1 │
└───────┴─────────────────┘

### 6-Left join
C'est une combinaison des lignes de deux tables avec un condition de jointure spécifique, mais qui garde toute les lignes de la table de gauche (1ière table) et les lignes correspondantes de la table de droite (2nd table).

**Mission :** Rédiger une requête qui permet d'avoir sur la table des ventes le nom des clients et le titres des articles.

In [ ]:
query ="""
SELECT 
    date,
    cust_id,
    client_name,
    item,
    item_name,
    qty,
    price,
    line_no,
    inv_no
FROM
    table_invoice
LEFT JOIN 
    table_client
    ON 
        cust_id = id_client
LEFT JOIN
    table_item
    ON
        item = code_item
"""
duckdb.sql(query)

┌─────────────────────┬─────────┬─────────────────────────────────────────┬────────────┬─────────────────────────────────┬───────┬────────┬─────────┬──────────┐
│        date         │ cust_id │               client_name               │    item    │            item_name            │  qty  │ price  │ line_no │  inv_no  │
│    timestamp_ns     │ varchar │                 varchar                 │  varchar   │             varchar             │ int64 │ double │  int64  │ varchar  │
├─────────────────────┼─────────┼─────────────────────────────────────────┼────────────┼─────────────────────────────────┼───────┼────────┼─────────┼──────────┤
│ 2025-01-01 00:00:00 │ fr133   │ L'Ambroisie                             │ rpo1979100 │ Pétrus 1979                     │     3 │ 5320.0 │       1 │ fac01_01 │
│ 2025-01-03 00:00:00 │ fr141   │ Assiette Champenoise                    │ rch2009100 │ Deutz 2009                      │    10 │  210.0 │       1 │ fac01_02 │
│ 2025-01-05 00:00:00 │ fr116   │ 

# Les window functions
Les fonctions fenêtres permettent d'effectuer des calculs sur un ensemble de lignes tout en conservant les lignes d'origines.\

Pour rappel l'ordre des clauses est le suivant 'PARTITION BY' (divise en groupe) puis 'ORDER BY' (classe les lignes) puis 'ROWS BETWEEN' (définie les lignes dans la plage). 

### 1-La clause OVER()
Cette clause s'utilise avec un fonction d'aggrégation ou avec des fonctions comme ROW_NUMBER, RANK(), DENSE_RANK() ou LAG().

**OVER (ORDER BY...)**
Cette clause définie la manière dont les lignes d'une partition seront triées selon une colonne.

**Mission** : Ecrire une requête qui permet de connaitre les ventes cumulées par semaine.

In [29]:
query = """
-- Création d'un CTE pour avoir les ventes par semaine --
WITH weekly_sales AS (
    SELECT 
        WEEK(date) AS week_no,
        SUM(qty * price) AS weekly_sales
    FROM 
        table_invoice
    GROUP BY 
        WEEK(date)
)

-- Requête finale pour obtenir les ventes cumulées par semaine --
SELECT 
    week_no,
    weekly_sales,
    SUM(weekly_sales) OVER (ORDER BY week_no) AS cumulated_sales
FROM 
    weekly_sales
"""
duckdb.sql(query)


┌─────────┬───────────────────┬───────────────────┐
│ week_no │   weekly_sales    │  cumulated_sales  │
│  int64  │      double       │      double       │
├─────────┼───────────────────┼───────────────────┤
│       1 │           18804.0 │           18804.0 │
│       2 │ 820.9000000000001 │           19624.9 │
│       3 │             672.0 │           20296.9 │
│       4 │           14364.3 │           34661.2 │
│       5 │            2784.0 │           37445.2 │
│       6 │             481.5 │           37926.7 │
│       7 │           16705.4 │           54632.1 │
│       8 │            8670.0 │           63302.1 │
│       9 │            4120.5 │           67422.6 │
│      10 │            1976.9 │           69399.5 │
│      11 │            8309.2 │           77708.7 │
│      12 │            5706.4 │ 83415.09999999999 │
│      13 │            9072.0 │ 92487.09999999999 │
│      14 │          20478.65 │         112965.75 │
│      15 │ 5101.799999999999 │         118067.55 │
│      16 │ 

**OVER (ORDER BY ... ROWS BETWEEN ...)**\
Cette commande détermine précisement quelles lignes sont à inclure dans le calcul.\
On utilise 'PRECEDING', 'FOLLOWING', 'UNBOUNDED' et 'CURRENT ROW'.

**Mission** : A partir le la table des factures calculer les ventes moyennes sur les 3 derniers jours facturées. 

In [ ]:
query = '''

-- calculer les ventes journalières avec un CTE
WITH cte1 AS (
    SELECT 
        date,
        SUM(qty*price) AS sales
    FROM 
        table_invoice
    GROUP BY
        date
)

-- requête finale 
SELECT
    date,
    sales,
    AVG(sales) OVER (ORDER BY date
                            ROWS BETWEEN 3 PRECEDING and CURRENT ROW) AS average_sales_3d
FROM 
    cte1
'''
duckdb.sql(query)

┌─────────────────────┬────────────────────┬────────────────────┐
│        date         │       sales        │  average_sales_3d  │
│    timestamp_ns     │       double       │       double       │
├─────────────────────┼────────────────────┼────────────────────┤
│ 2025-01-01 00:00:00 │            15960.0 │            15960.0 │
│ 2025-01-03 00:00:00 │             2100.0 │             9030.0 │
│ 2025-01-05 00:00:00 │              744.0 │             6268.0 │
│ 2025-01-09 00:00:00 │ 277.90000000000003 │           4770.475 │
│ 2025-01-10 00:00:00 │              543.0 │            916.225 │
│ 2025-01-15 00:00:00 │              672.0 │            559.225 │
│ 2025-01-22 00:00:00 │            14364.3 │ 3964.2999999999997 │
│ 2025-01-30 00:00:00 │             2784.0 │           4590.825 │
│ 2025-02-05 00:00:00 │              481.5 │            4575.45 │
│ 2025-02-13 00:00:00 │             1085.4 │             4678.8 │
│          ·          │                ·   │                ·   │
│         

**OVER (PARTITION BY ...)**\
Cette clause permet de diviser les données en sous-ensembles (=partition) selon les valeurs dans les colonnes spécifiées.

**Mission** : Rédiger une requête qui donne les ventes totales par client et le total des ventes de leur pay respectif.

In [ ]:
query = """

-- Création d'un CTE pour ajouter le nom du client et son pays dans la table des ventes --
WITH CTE AS (
    SELECT
        cust_id,
        client_name,
        SUM (qty * price) AS sales_cust,
        country
    FROM 
        table_invoice
    LEFT JOIN 
        table_client
    ON
        cust_id = id_client
    GROUP BY
        cust_id,
        client_name,
        country
)
SELECT 
    *,
    SUM(sales_cust) 
        OVER (PARTITION BY country) AS sales_country
FROM
    CTE
"""
duckdb.sql(query)

┌─────────┬─────────────────────────────────────────┬────────────────────┬─────────────┬───────────────┐
│ cust_id │               client_name               │     sales_cust     │   country   │ sales_country │
│ varchar │                 varchar                 │       double       │   varchar   │    double     │
├─────────┼─────────────────────────────────────────┼────────────────────┼─────────────┼───────────────┤
│ eu115   │ Le Bernardin                            │             6190.4 │ Etats-Unis  │        6190.4 │
│ ja226   │ Sézanne                                 │            16460.4 │ Japon       │       16460.4 │
│ ru232   │ The ledbury                             │             1895.0 │ Royaume-Uni │        1895.0 │
│ fr134   │ Arpège                                  │            19559.4 │ France      │      71006.25 │
│ fr121   │  Le 1947 à Cheval Blanc                 │            14240.0 │ France      │      71006.25 │
│ fr128   │  Mirazur                                │  

### 2-La clause LAG()

LAG (*expression* [,*decalage*[,*valeur_par_défaut*]])

La commande 'LAG' est utilisée dans un 'SELECT' avec la commande 'OVER()'.\
Elle permet d'accéder à une valeur provenant d'une ligne précédente dans une colonne définie.

**Mission** :

# La commande GROUP BY

### 1-GROUP BY et HAVING

La commande HAVING permet de filtrer les lignes produites par un GROUP BY. Elle remplace le WHERE qui ne peut pas être utilisé.

**Mission**

### 2-GROUPING SETS
Cette commande permet de réaliser des sous-groupes dans une requête comme si nous enchaînions plusieurs 'GROUP BY'.

**Mission** : Comment se répartissent les ventes par pays du client dans chaque région viticole ? Je veux aussi voir les ventes totales par région.

In [ ]:
query ='''
-- CTE pour ajouter le pays (client) et la région (produit)
WITH cte AS(
    SELECT *  
    FROM 
            table_invoice AS r
    LEFT JOIN 
            table_client AS l
    ON
            r.cust_id = l.id_client
    LEFT JOIN
            table_item AS ll
    ON 
        r.item = ll.code_item
)

SELECT  region,
        country,
        SUM(qty * price) AS vente
FROM 
    cte
GROUP BY GROUPING SETS ((region, country),region)
ORDER BY region, country

'''
duckdb.sql(query)

┌────────────────┬────────────┬────────────────────┐
│     region     │  country   │       vente        │
│    varchar     │  varchar   │       double       │
├────────────────┼────────────┼────────────────────┤
│ Champagne      │ Chine      │             4920.0 │
│ Champagne      │ France     │            11143.0 │
│ Champagne      │ Italie     │             2689.0 │
│ Champagne      │ NULL       │            18752.0 │
│ Margaux        │ France     │             2184.0 │
│ Margaux        │ NULL       │             2184.0 │
│ Pauillac       │ Italie     │            11866.8 │
│ Pauillac       │ Japon      │ 14240.400000000001 │
│ Pauillac       │ NULL       │            26107.2 │
│ Pessac Leognan │ Argentine  │             2400.0 │
│       ·        │   ·        │                ·   │
│       ·        │   ·        │                ·   │
│       ·        │   ·        │                ·   │
│ Saint Estèphe  │ France     │             7228.5 │
│ Saint Estèphe  │ Italie     │             24

# Autres commandes


### 1-La commande CAST:
CAST(*expression* AS *type*)

Cette commande, utilisée dans un 'SELECT', permet de définir ou modifier le type des données d'une colonne.

**Mission** : Ajouter dans la table des factures une colonne avec les ventes arrondies à l'unité.

In [ ]:
query = '''
SELECT 
    date,
    cust_id,
    item,
    qty,
    price,
    CAST(qty*price AS integer) AS total_sales
FROM 
    table_invoice
'''
duckdb.sql(query)

┌─────────────────────┬─────────┬────────────┬───────┬────────┬─────────────┐
│        date         │ cust_id │    item    │  qty  │ price  │ total_sales │
│    timestamp_ns     │ varchar │  varchar   │ int64 │ double │    int32    │
├─────────────────────┼─────────┼────────────┼───────┼────────┼─────────────┤
│ 2025-01-01 00:00:00 │ fr133   │ rpo1979100 │     3 │ 5320.0 │       15960 │
│ 2025-01-03 00:00:00 │ fr141   │ rch2009100 │    10 │  210.0 │        2100 │
│ 2025-01-05 00:00:00 │ fr116   │ rpo2013100 │     5 │  148.8 │         744 │
│ 2025-01-09 00:00:00 │ fr133   │ bsa2012100 │     7 │   39.7 │         278 │
│ 2025-01-10 00:00:00 │ fr140   │ bsa2018100 │     3 │  181.0 │         543 │
│ 2025-01-15 00:00:00 │ fr124   │ rma2021101 │     4 │  168.0 │         672 │
│ 2025-01-22 00:00:00 │ it236   │ rpa2019100 │    12 │  988.9 │       11867 │
│ 2025-01-22 00:00:00 │ it236   │ res2017100 │    15 │  166.5 │        2498 │
│ 2025-01-30 00:00:00 │ eu115   │ bpe2020100 │    16 │  174.0 │ 

### 2-La commande CASE WHEN
CASE \
WHEN *condition 1* THEN *resultat 1*\
  WHEN *condition 2* THEN *resultat 2*\
     ELSE *resultat 3*\
     END AS *expression*

 Cette commande permet d'ajouter une logique conditionelle dans les requêtes et retourne différentes valeurs basées sur des conditions spécifiques.\
 Elle est surtout utilisée dans un 'SELECT' mais elle peut aussi être utilisée dans une aggrégation.
 
 **Mission 1** : Créer une requête qui regroupe les ventes par pays en deux zones géographiques ; 'local' (cad France) et 'export' pour les autres pays.

In [ ]:
query = '''
-- ajouter le pays aux ventes et ajouter la colonne spécifique pour local / export
SELECT country, 
        SUM((qty * price)) AS sales,
        CASE 
            WHEN country = 'France' THEN 'local'
            ELSE 'export'
        END AS geo 
FROM 
        table_invoice AS r
LEFT JOIN 
        table_client AS l
ON
        r.cust_id = l.id_client

GROUP BY
         country
'''
duckdb.sql(query)

┌─────────────┬──────────┬─────────┐
│   country   │  sales   │   geo   │
│   varchar   │  double  │ varchar │
├─────────────┼──────────┼─────────┤
│ Chine       │  24672.0 │ export  │
│ France      │ 71006.25 │ local   │
│ Italie      │  27983.5 │ export  │
│ Argentine   │   4830.0 │ export  │
│ Japon       │  16460.4 │ export  │
│ Etats-Unis  │   6190.4 │ export  │
│ Royaume-Uni │   1895.0 │ export  │
└─────────────┴──────────┴─────────┘

**Mission 2** : Pour chaque client, calculer la somme de ses ventes par pays. 

In [ ]:
query = '''
SELECT cust_id, 
            SUM(
                CASE
                    WHEN country ='France' THEN qty * price
                    ELSE 0
                END) AS ventes_FR,
           SUM(
                CASE
                    WHEN country ='Italie' THEN qty * price
                    ELSE 0
                END) AS ventes_IT  ,      
            SUM(
                CASE
                    WHEN country ='Chine' THEN qty * price
                    ELSE 0
                END) AS vente_CH   ,    
           SUM(
                CASE
                    WHEN country ='Japon' THEN qty * price
                    ELSE 0
                END) AS vente_JP,
           SUM(
                CASE
                    WHEN country ='Etats-Unis' THEN qty * price
                    ELSE 0
                END) AS vente_US,
           SUM(
                CASE
                    WHEN country ='Royaume-Uni' THEN qty * price
                    ELSE 0
                END) AS vente_UK,      
            country,
FROM 
            table_invoice AS r
LEFT JOIN 
            table_client AS l
ON
            r.cust_id = l.id_client
GROUP BY 
        cust_id,
        country
'''
duckdb.sql(query)

┌─────────┬────────────────────┬───────────┬──────────┬──────────┬──────────┬──────────┬─────────────┐
│ cust_id │     ventes_FR      │ ventes_IT │ vente_CH │ vente_JP │ vente_US │ vente_UK │   country   │
│ varchar │       double       │  double   │  double  │  double  │  double  │  double  │   varchar   │
├─────────┼────────────────────┼───────────┼──────────┼──────────┼──────────┼──────────┼─────────────┤
│ fr140   │ 3143.3999999999996 │       0.0 │      0.0 │      0.0 │      0.0 │      0.0 │ France      │
│ it223   │                0.0 │   10705.2 │      0.0 │      0.0 │      0.0 │      0.0 │ Italie      │
│ fr133   │            21446.2 │       0.0 │      0.0 │      0.0 │      0.0 │      0.0 │ France      │
│ fr124   │            5057.75 │       0.0 │      0.0 │      0.0 │      0.0 │      0.0 │ France      │
│ ar102   │                0.0 │       0.0 │      0.0 │      0.0 │      0.0 │      0.0 │ Argentine   │
│ it236   │                0.0 │   17278.3 │      0.0 │      0.0 │      0

### 3-La commande ROUND()

ROUND (*valeur_numérique*, *nb_décimale*)

Cette commande permet d'arrondir une valeur numérique en choisissant le nombre de décimale.

**Mission** : Calculer les ventes par facture avec une décimale.

In [ ]:
query = """
SELECT inv_no,
    cust_id,
    ROUND(SUM(qty*price),1) AS total_sales
FROM 
    table_invoice
GROUP
    BY inv_no,
    cust_id
ORDER BY 
    inv_no
"""
duckdb.sql(query)

┌──────────┬─────────┬─────────────┐
│  inv_no  │ cust_id │ total_sales │
│ varchar  │ varchar │   double    │
├──────────┼─────────┼─────────────┤
│ fac01_01 │ fr133   │     15960.0 │
│ fac01_02 │ fr141   │      2100.0 │
│ fac01_03 │ fr116   │       744.0 │
│ fac01_04 │ fr133   │       277.9 │
│ fac01_05 │ fr140   │       543.0 │
│ fac01_06 │ fr124   │       672.0 │
│ fac01_07 │ it236   │     14364.3 │
│ fac01_08 │ eu115   │      2784.0 │
│ fac02_01 │ fr118   │       481.5 │
│ fac02_02 │ fr133   │      1085.4 │
│    ·     │   ·     │         ·   │
│    ·     │   ·     │         ·   │
│    ·     │   ·     │         ·   │
│ fac04_09 │ it223   │      7778.0 │
│ fac04_10 │ fr134   │      1150.0 │
│ fac04_11 │ fr140   │       645.0 │
│ fac04_12 │ fr143   │       925.0 │
│ fac04_13 │ ar102   │      4830.0 │
│ fac04_14 │ it236   │      1375.0 │
│ fac04_15 │ fr134   │      3240.0 │
│ fac05_01 │ fr121   │      2868.0 │
│ fac05_02 │ it236   │      1539.0 │
│ fac05_03 │ fr134   │     10620.0 │
├

### 4-La commande LIKE
La commande LIKE est utilisée avec WHERE et permet de sélectionner des valeurs avec les caractères spéciaux '%' qui remplace n'importe quel nombre de caractères et '_' qui représente un seul caractère. 

**Mission** : Dans la table des produits, combien de château d'Yquem il est possible de vendre ?

In [ ]:
query = """
SELECT *
FROM table_item
WHERE item_name LIKE '%Yquem%'
"""
duckdb.sql(query)

┌────────────┬──────────────────────┬───────┬────────────┬───────────┬─────────┐
│ code_item  │      item_name       │ year  │ base_price │  region   │  type   │
│  varchar   │       varchar        │ int64 │   double   │  varchar  │ varchar │
├────────────┼──────────────────────┼───────┼────────────┼───────────┼─────────┤
│ bsa2018100 │ Château d'Yquem 2018 │  2018 │      169.0 │ Sauternes │ blanc   │
│ bsa2019100 │ Château d'Yquem 2019 │  2019 │      390.0 │ Sauternes │ blanc   │
│ bsa2020100 │ Château d'Yquem 2020 │  2020 │      430.0 │ Sauternes │ blanc   │
└────────────┴──────────────────────┴───────┴────────────┴───────────┴─────────┘

***************************************************************************************************

# 2-Solutions pratiques avec les clauses 'join'

### 2.1 - Variation hebdomadaire en pourcentage des clients avec 'Self-Join'.
Contexte :\
Évaluer l'évolution des clients est essentiel pour suivre un indicateur clé comme le nombre de clients actifs.\
Mission :\
Déterminer la variation en pourcentage d'une semaine à l'autre pour les clients hebdomadaire actifs.

Pour rappel :\
Le 'self join' est une opération de jointure d'une table avec elle-même qui nécessite d'utiliser des alias dans 'SELECT', 'FROM' et 'JOIN' pour distinguer les deux instances.


In [ ]:
query = """

-- Créer un CTE avec par facture la semaine et le code client --
WITH cte AS (
    SELECT 
        WEEK(date) AS week,
        cust_id,
        inv_no
    FROM table_invoice
    GROUP BY date, cust_id, inv_no
    ORDER BY week
),
    cte2 AS (
    SELECT 
        week,
        COUNT(DISTINCT(cust_id)) AS nb_cust
    FROM cte
    GROUP BY(week)
)

-- Requête finale à l'aide de cte --

SELECT 
    l.week AS previous_week,
    l.nb_cust AS prev_nb_cust,
    r.week AS current_week,
    r.nb_cust AS curr_nb_cust,
    ROUND(100*((curr_nb_cust / prev_nb_cust)-1),2) AS percent_var_nb_cust
FROM
    cte2 AS l
JOIN
    cte2 AS r
ON 
    l.week = r.week + 1

ORDER BY previous_week

"""
duckdb.sql(query)

┌───────────────┬──────────────┬──────────────┬──────────────┬─────────────────────┐
│ previous_week │ prev_nb_cust │ current_week │ curr_nb_cust │ percent_var_nb_cust │
│     int64     │    int64     │    int64     │    int64     │       double        │
├───────────────┼──────────────┼──────────────┼──────────────┼─────────────────────┤
│             2 │            2 │            1 │            3 │                50.0 │
│             3 │            1 │            2 │            2 │               100.0 │
│             4 │            1 │            3 │            1 │                 0.0 │
│             5 │            1 │            4 │            1 │                 0.0 │
│             6 │            1 │            5 │            1 │                 0.0 │
│             7 │            2 │            6 │            1 │               -50.0 │
│             8 │            3 │            7 │            2 │              -33.33 │
│             9 │            3 │            8 │            3 │   

### 2.2 - Produits jamais commandés avec 'left join' et 'cross join':
Mission : \
Lister pour chaque client les produits du catalogue qui n'ont pas été commandés sur l'année.

Pour rappel :\
Un 'cross join' est un produit cartésien. Chaque ligne de la première table est combinée avec les lignes de la seconde table.\
Un 'left join' est une combinaison des lignes de deux tables avec un condition de jointure sécifique. La jointure garde toute les lignes de la table de gauche (1ière table) et les lignes correspondantes de la table de droite (2nd table).

In [ ]:
query = """

-- définir une table des ventes par client et articles --
WITH cte_ventes AS(
    SELECT 
        cust_id,
        item,
        SUM(qty * price) AS annual_sales
    FROM
        table_invoice
    GROUP BY
        cust_id, item        
)

-- Requete principale qui liste les clients et les produits jamais commandés --

SELECT 
    l.cust_id AS cust_code,
    l.name AS cust_name, 
    r.code_item AS item_code,
    r.item_name AS item_name

FROM 
    table_client AS l
    -- permet de créer une table de tous les clients avec tous les produits --
    CROSS JOIN 
    table_item AS r
LEFT JOIN
    cte_ventes AS v
ON
    (l.cust_id = v.cust_id) AND(r.code_item = v.item) 
GROUP BY 
    l.cust_id,l.name, r.code_item, r.item_name
HAVING 
    SUM(annual_sales) IS NULL
ORDER BY
    l.cust_id, r.code_item
"""
duckdb.sql(query)

BinderException: Binder Error: Table "l" does not have a column named "cust_id"

### 2.3 - Produits jamais commandés avec 'cross join' et 'full outer join':
Meme objectif que précédement avec un FULL OUTER JOIN.

Pour rappel :\
Le résultat d'un 'full outer join' retourne toutes les lignes des deux tables avec des valeurs NULL là où il n'y a pas de correspondances.

In [ ]:
query = """
-- premier cte pour récuperer les clients et les produits effectivement vendus --
WITH 
cte1 AS (
    SELECT 
        cust_id,
        item,
    FROM 
        table_invoice
    GROUP BY 
        cust_id,
        item
    ORDER BY
        cust_id,
        item
),

-- second cte pour créer une liste des clients avec tous les produits --
cte2 AS (
    SELECT 
        cust_id,
        code_item,
        name,
        item_name
    FROM 
        table_client
    CROSS JOIN 
        table_item
    ORDER BY
        cust_id,
        code_item
)

-- on réalise la jointure des deux cte en selectionnant les clients et produits absent du premier cte --
    SELECT
        l.cust_id,
        l.name,
        l.code_item,
        l.item_name
    FROM
        cte2 AS l
    FULL OUTER JOIN
        cte1 AS r
    ON
        (l.cust_id = r.cust_id)
         AND 
        (l.code_item = r.item)
    WHERE
        r.cust_id IS NULL
    ORDER BY
        l.cust_id,
        l.code_item
"""

duckdb.sql(query)

BinderException: Binder Error: Referenced column "cust_id" not found in FROM clause!
Candidate bindings: "table_client.country", "table_item.code_item", "table_item.base_price", "table_client.client_name", "table_item.item_name"

### 2.4 - Les ventes par pays et par type de vin avec 'inner join'
Contexte :\
On cherche à savoir si il existe une relation entre le pays du client et la région du vin de bordeaux.\
Mission :\
Extraire les région de bordeaux vendu pour chaque pays de vente et le nombre de bouteilles.

Pour rappel:\
Un 'inner join' permet de combiner les lignes de deux tables en se basant sur une condition de jointure spécifique. Il renvoie uniquement les lignes où il y a une correspondance dans les deux tables. C'est un 'crossjoin' avec une condition de correspondance.

In [ ]:
query = """
SELECT 
    r.country,
    rr.region,
    SUM(l.qty) AS total_qty
FROM
    table_invoice AS l
INNER JOIN
    table_client AS r
INNER JOIN
    table_item AS rr
ON
    l.cust_id = r.cust_id
ON
    l.item = rr.code_item
GROUP BY
    r.country,
    rr.region
Order BY 
    rr.region,
    total_qty DESC
"""
duckdb.sql(query)

BinderException: Binder Error: Referenced table "r" not found!
Candidate tables: "l"

### 2.5- Commandes clients maintenues ou non d'un mois sur l'autre avec un 'self join' et un 'full ouetr join'
Mission n°1 :\
Écrire une requête qui permet d'obtenir le nombre de clients qui ont passé une commande le mois précédent et le mois présent.

In [ ]:
query = """

-- créer une sous_table en provenance des ventes avec numéro client, nom et le mois de la facture--

WITH cte AS (
    SELECT 
        cust_id,
        MONTH(date) as month,
    FROM table_invoice
    GROUP BY cust_id, month
)

-- requete principale avec un SELF JOIN --
SELECT 
    l.month,
    COUNT(DISTINCT(l.cust_id)) AS retained_client
FROM 
    cte AS l
JOIN
    cte AS r
ON 
    l.cust_id = r.cust_id
    AND
    l.month = r.month + 1
GROUP BY 
    l.month
ORDER BY
    l.month

"""
duckdb.sql(query)

┌───────┬─────────────────┐
│ month │ retained_client │
│ int64 │      int64      │
├───────┼─────────────────┤
│     2 │               2 │
│     3 │               3 │
│     4 │               3 │
│     5 │               2 │
└───────┴─────────────────┘

Mission n°2 :\
Rédiger une requête qui donne le nombre de clients du mois dernier qui n'ont pas achetés ce mois-ci.

In [ ]:
query = """

-- créer une sous_table en provenance des ventes avec numéro client, nom et le mois de la facture--

WITH cte AS (
    SELECT 
        cust_id,
        MONTH(date) as month,
    FROM table_invoice
    GROUP BY cust_id, month
)

SELECT
    l.month,
    COUNT(DISTINCT(r.cust_id)) AS left_client
FROM
    cte AS l
FULL OUTER JOIN
    cte AS r
ON
    l.cust_id = r.cust_id
    AND 
    l.month = r.month + 1
WHERE 
    l.cust_id IS NULL
GROUP BY 
    l.month
ORDER BY 
    l.month

"""
duckdb.sql(query)